In [ ]:
pip install torch_geometric

In [ ]:
pip install torch_scatter

# Imports

In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.datasets import KarateClub
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader


# Dataset & Dataloader

In [2]:
dataset = KarateClub()

In [3]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# GNN

In [67]:
class GNN(torch.nn.Module):
    def __init__(self, node_features, num_classes):
        super().__init__()
        self.conv1 = GCNConv(node_features, 32)
        self.conv2 = GCNConv(32, num_classes)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x)
        
        x = self.conv2(x, edge_index)
        prob = F.log_softmax(x, dim=1)

        return prob

# Device

In [68]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device used:', device)

Device used: cpu


# Model Initialization

In [69]:
node_features = dataset.num_node_features
num_classes = dataset.num_classes
node_features, num_classes

(34, 4)

In [70]:
model = GNN(node_features, num_classes).to(device)

In [100]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train one epoch

In [101]:
def train_one_epoch(dataloader, model, optimizer):
    model.train()
    total_loss = 0.

    for batch in dataloader:
        sample = batch.to(device)
        optimizer.zero_grad()
        out = model(sample)
        
        loss = F.nll_loss(out[sample.train_mask], sample.y[sample.train_mask])
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    return total_loss

# Main Run

In [117]:
epochs = 500

In [118]:
for epoch in range(epochs):
    train_loss = train_one_epoch(dataloader, model, optimizer)
    print('Epoch:', epoch, 'train_loss:', train_loss)

Epoch: 0 train_loss: 0.04450872167944908
Epoch: 1 train_loss: 0.047739945352077484
Epoch: 2 train_loss: 0.040236398577690125
Epoch: 3 train_loss: 0.08639806509017944
Epoch: 4 train_loss: 0.03258740156888962
Epoch: 5 train_loss: 0.04422576352953911
Epoch: 6 train_loss: 0.0523206926882267
Epoch: 7 train_loss: 0.10172517597675323
Epoch: 8 train_loss: 0.04142517223954201
Epoch: 9 train_loss: 0.052501074969768524
Epoch: 10 train_loss: 0.04089485853910446
Epoch: 11 train_loss: 0.08233557641506195
Epoch: 12 train_loss: 0.038822241127491
Epoch: 13 train_loss: 0.0587797537446022
Epoch: 14 train_loss: 0.03674997389316559
Epoch: 15 train_loss: 0.036178432404994965
Epoch: 16 train_loss: 0.11324281990528107
Epoch: 17 train_loss: 0.03685916215181351
Epoch: 18 train_loss: 0.04848182573914528
Epoch: 19 train_loss: 0.05629263073205948
Epoch: 20 train_loss: 0.04687010869383812
Epoch: 21 train_loss: 0.029830701649188995
Epoch: 22 train_loss: 0.04352683946490288
Epoch: 23 train_loss: 0.03549201413989067
E

# Model evaluation

In [119]:
model.eval()
eval_data = next(iter(dataloader))
pred = model(eval_data).argmax(dim=1)

In [120]:
test_mask = ~eval_data.train_mask

In [121]:
correct = (pred[test_mask] == eval_data.y[test_mask]).sum()
acc = int(correct) / int(test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7000
